# Защита персональных данных клиентов

Стоит задача защитить данные клиентов страховой компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и обосновать корректность его работы.

Данные нужно защитить так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
display(df.info())
display(df.shape)
display(df.head())
display(df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

(5000, 5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


## Умножение матриц

Подберем оптимальные веса для признаков X и XP путем преобразования формулы обучения

$w = (X^T X)^{-1} X^T$

$w'= ((XP)^{T}XP)^{-1}(XP)^{T} = (P^TX^TXP)^{-1} P^T X^T= P^{-1}(X^TX)^{-1}(P^T)^{-1}P^{T}X^T = P^{-1}(X^TX)^{-1}X^T = P^{-1} w$

Сделаем предсказания. Для этого нужно матрицу признаков умножить на веса.

$a = Xw$

$a' = XPw' = XPP^{-1}w = Xw = a$

Мы видим, что предсказания получились одинаковые для матрицы признаков Х и для матрицы признаков ХP

**Алгоритм**

1. Выделим из исходной таблицы целевой признак
2. Обучим модель и посчитаем метрику R2
3. Преобразуем таблицу признаков в матрицу
4. Сформируем случайную обратимую матрицу размером 4х4 (по количеству признаков)
5. Получим новую таблицу признакв путем умножения матрицы признаков на случайную матрицу
6. Обучим модель и посчитаем метрику R2

**Обоснование**

Ранее теоретически путем преобразования формулы обучения было доказано, что после умножения матрицы признаков на обратимую матрицу предсазания не изменяются. Таким образом, качество модели не изменится.

## Проверка алгоритма

In [3]:
# Выделим из исходной таблицы целевой признак
target = df['Страховые выплаты']
features = df.drop(['Страховые выплаты'] , axis=1)

# Обучим модель и посчитаем метрику R2
model = LinearRegression()
model.fit(features, target)
print(model.score(features, target))

0.42494550286668


In [12]:
# Преобразуем таблицу признаков в матрицу
features_matrix = features.values

# Сформируем случайную обратимую матрицу размером 4х4 (по количеству признаков)
try:
    data_random = np.random.normal(size = (4,4)) 
except numpy.linalg.LinAlgError: 
    data_random = np.random.normal(size = (4,4)) 
    
# Получим новую таблицу признакв путем умножения матрицы признаков на случайную матрицу
features_new = pd.DataFrame(features_matrix @ data_random)

# Обучим модель и посчитаем метрику R2
model.fit(features_new, target)
print(model.score(features_new, target))

0.42494550286666477


**Вывод**

Мы убедились, что после умножения матрицы признаков на случайную матрицу качество модели не изменилось, т.к. метрика R2 в обоих случаях равна. Персональные данные клиентов защищены без ущерба для работы модели.